# Neural Graph Learning and Graph Regularization

In this tutorial, we will be creating a graph regularized version for a topic classification task. The task is to classify paper depending on their content. However in order to do so, we will also use the information encoded in the citation network that relates documents among each other. Of course, we do know that this kind of information is indeed powerful as papers belonging to the same subject tend to reference each other.   

### Load Dataset

For this tutorial we will be using the Cora dataset available in the stellargraph library 

In [1]:
from stellargraph import datasets

2023-03-22 11:15:24.593253: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-22 11:15:24.593280: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-22 11:15:25.657677: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-03-22 11:15:25.657811: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-22 11:15:25.657821: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-22 11:15:25.657840: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running 

In [2]:
dataset = datasets.Cora()

In [3]:
%config Completer.use_jedi = False

In [4]:
dataset.download()

In [5]:
label_index = {
      'Case_Based': 0,
      'Genetic_Algorithms': 1,
      'Neural_Networks': 2,
      'Probabilistic_Methods': 3,
      'Reinforcement_Learning': 4,
      'Rule_Learning': 5,
      'Theory': 6,
  }

In [6]:
G, labels = dataset.load()

We now create the Dataset object where we will both include information of the targeted sample (node) and its neighbors. In the following we will also allow to control the number of labelling instances to be used, in order to reproduce and evaluate the classification performance in a semi-supervised setting. 

In [7]:
import numpy as np
from sklearn import preprocessing, feature_extraction, model_selection

In [8]:
import tensorflow as tf
from tensorflow.train import Example, Features, Feature, Int64List, BytesList, FloatList

In [9]:
GRAPH_PREFIX="NL_nbr"

In [10]:
def _int64_feature(*value):
    """Returns int64 tf.train.Feature from a bool / enum / int / uint."""
    return Feature(int64_list=Int64List(value=list(value)))

def _bytes_feature(value):
    """Returns bytes tf.train.Feature from a string."""
    return Feature(
        bytes_list=BytesList(value=[value.encode('utf-8')])
    )

def _float_feature(*value):
    return Feature(float_list=FloatList(value=list(value)))


In [11]:
from functools import reduce
from typing import List, Tuple
import pandas as pd
import six

def addFeatures(x, y):
    res = Features()
    res.CopyFrom(x)
    res.MergeFrom(y)
    return res

def neighborFeatures(features: Features, weight: float, prefix: str):
    data = {f"{prefix}_weight": _float_feature(weight)}
    for name, feature in six.iteritems(features.feature):
        data[f"{prefix}_{name}"] = feature 
    return Features(feature=data)

def neighborsFeatures(neighbors: List[Tuple[Features, float]]):
    return reduce(
        addFeatures, 
        [neighborFeatures(sample, weight, f"{GRAPH_PREFIX}_{ith}") for ith, (sample, weight) in enumerate(neighbors)],
        Features()
    )

def getNeighbors(idx, adjMatrix, topn=5):
    weights = adjMatrix.loc[idx]
    return weights[weights>0].sort_values(ascending=False).head(topn).to_dict()
    

def semisupervisedDataset(G, labels, ratio=0.2, topn=5):
    n = int(np.round(len(labels)*ratio))
    
    labelled, unlabelled = model_selection.train_test_split(
        labels, train_size=n, test_size=None, stratify=labels
    )
    
    adjMatrix = pd.DataFrame.sparse.from_spmatrix(G.to_adjacency_matrix(), index=G.nodes(), columns=G.nodes())
    
    features = pd.DataFrame(G.node_features(), index=G.nodes())
    
    dataset = {
        index: Features(feature = {
            #"id": _bytes_feature(str(index)), 
            "id": _int64_feature(index),
            "words": _float_feature(*[float(x) for x in features.loc[index].values]), 
            "label": _int64_feature(label_index[label])
        })
        for index, label in pd.concat([labelled, unlabelled]).items()
    }
    
    trainingSet = [
        Example(features=addFeatures(
            dataset[exampleId], 
            neighborsFeatures(
                [(dataset[nodeId], weight) for nodeId, weight in getNeighbors(exampleId, adjMatrix, topn).items()]
            )
        ))
        for exampleId in labelled.index
    ]
    
    testSet = [Example(features=dataset[exampleId]) for exampleId in unlabelled.index]

    serializer = lambda _list: [e.SerializeToString() for e in _list]
    
    return serializer(trainingSet), serializer(testSet)


We split the dataset into a training set and a test set

In [12]:
trainingSet, testSet = semisupervisedDataset(G, labels)

In [13]:
import tensorflow as tf

In [14]:
from tensorflow.data import Dataset

In [15]:
vocabularySize = 1433

In [16]:
neighbors=2
defaultWord = tf.constant(0, dtype=tf.float32, shape=[vocabularySize])

def parseExample(example, training=True):
    schema = {
        'words': tf.io.FixedLenFeature([vocabularySize], tf.float32, default_value=defaultWord),
        'label': tf.io.FixedLenFeature((), tf.int64, default_value=-1)
    }
    
    if training is True:
        for i in range(neighbors):
            name = f"{GRAPH_PREFIX}_{i}"
            schema[f"{name}_weight"] = tf.io.FixedLenFeature([1], tf.float32, default_value=[0.0])
            schema[f"{name}_words"] = tf.io.FixedLenFeature([vocabularySize], tf.float32, default_value=defaultWord)
    
    features = tf.io.parse_single_example(example, schema)
    
    label = features.pop("label")
    return features, label

In [17]:
def sampleGenerator(dataset):
    def wrapper():
        for example in dataset:
            yield example
    return wrapper
            
myTrain = Dataset \
    .from_generator(sampleGenerator(trainingSet), output_types=tf.string, output_shapes=()) \
    .map(lambda x: parseExample(x, True))

myTest = Dataset \
    .from_generator(sampleGenerator(testSet), output_types=tf.string, output_shapes=()) \
    .map(lambda x: parseExample(x, False))


In [18]:
for features, labels in myTrain.batch(10).take(1):
    print(features)
    print(labels)

{'NL_nbr_0_weight': <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>, 'NL_nbr_0_words': <tf.Tensor: shape=(10, 1433), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, 'NL_nbr_1_weight': <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.]], dtype=float32)>, 'NL_nbr_1_words': <tf.Tensor: shape=(10, 1433), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
     

2023-03-22 11:15:46.441988: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-03-22 11:15:46.459178: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz


In [19]:
for features, labels in myTest.batch(10).take(1):
    print(features)
    print(labels)

{'words': <tf.Tensor: shape=(10, 1433), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>}
tf.Tensor([3 1 6 2 6 1 2 6 6 2], shape=(10,), dtype=int64)


### Creating the model 

We now create the model that we will use to classify the documents

In [20]:
layers = [50, 50]

In [21]:
"""Creates a functional API-based multi-layer perceptron model."""
def create_model(num_units):
    inputs = tf.keras.Input(
          shape=(vocabularySize,), dtype='float32', name='words'
    )

    # outputs = tf.keras.layers.Dense(len(label_index), activation='softmax')(inputs)

    cur_layer =  inputs

    for num_units in layers:
        cur_layer = tf.keras.layers.Dense(num_units, activation='relu')(cur_layer)
        cur_layer = tf.keras.layers.Dropout(0.8)(cur_layer)

    outputs = tf.keras.layers.Dense(len(label_index), activation='softmax')(cur_layer)

    return tf.keras.Model(inputs, outputs=outputs)

In [22]:
from tensorflow.keras.callbacks import TensorBoard

#### Vanilla Model

We first train a simple, vanilla version that does not use the citation network information

In [23]:
model = create_model([50, 50])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [24]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
words (InputLayer)           [(None, 1433)]            0         
_________________________________________________________________
dense (Dense)                (None, 50)                71700     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 357       
Total params: 74,607
Trainable params: 74,607
Non-trainable params: 0
_________________________________________________________

In [25]:
model.fit(myTrain.batch(128), epochs=200, verbose=1, validation_data=myTest.batch(128),
          callbacks=[TensorBoard(log_dir='/tmp/noRegularization')])

2023-03-22 11:15:46.613974: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2023-03-22 11:15:46.614008: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2023-03-22 11:15:46.614060: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


Epoch 1/200


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/functional.py:595: UserWarning: Input dict contained keys ['NL_nbr_0_weight', 'NL_nbr_0_words', 'NL_nbr_1_weight', 'NL_nbr_1_words'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


      4/Unknown - 0s 20ms/step - loss: 2.0418 - accuracy: 0.1475 

2023-03-22 11:15:47.021549: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2023-03-22 11:15:47.021587: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2023-03-22 11:15:47.025267: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2023-03-22 11:15:47.026507: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2023-03-22 11:15:47.028375: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /tmp/noRegularization/train/plugins/profile/2023_03_22_11_15_47
2023-03-22 11:15:47.029345: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to /tmp/noRegularization/train/plugins/profile/2023_03_22_11_15_47/5bd49021e10b.trace.json.gz
2023-03-22 11:15:47.031718: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /tmp/noRegularization/train/plugins/profile/

5/5 [==============================] - 1s 165ms/step - loss: 2.0492 - accuracy: 0.1407 - val_loss: 1.9435 - val_accuracy: 0.1325
Epoch 2/200
5/5 [==============================] - 0s 69ms/step - loss: 1.9879 - accuracy: 0.1636 - val_loss: 1.9347 - val_accuracy: 0.1787
Epoch 3/200
5/5 [==============================] - 0s 69ms/step - loss: 1.9847 - accuracy: 0.1693 - val_loss: 1.9272 - val_accuracy: 0.2281
Epoch 4/200
5/5 [==============================] - 0s 66ms/step - loss: 1.9697 - accuracy: 0.1513 - val_loss: 1.9202 - val_accuracy: 0.2710
Epoch 5/200
5/5 [==============================] - 0s 67ms/step - loss: 1.9637 - accuracy: 0.1907 - val_loss: 1.9140 - val_accuracy: 0.2913
Epoch 6/200
5/5 [==============================] - 0s 65ms/step - loss: 1.9211 - accuracy: 0.2356 - val_loss: 1.9088 - val_accuracy: 0.2964
Epoch 7/200
5/5 [==============================] - 0s 70ms/step - loss: 1.9173 - accuracy: 0.2219 - val_loss: 1.9035 - val_accuracy: 0.2973
Epoch 8/200
5/5 [==============

#### Graph Regularized Version

We now create the graph-regularized version that uses the citation network information

In [26]:
base_model = create_model([50, 50])

In [27]:
import neural_structured_learning as nsl

In [28]:
graph_reg_config = nsl.configs.make_graph_reg_config(
    max_neighbors=2,
    multiplier=0.1,
    distance_type=nsl.configs.DistanceType.L2,
    sum_over_axis=-1)
graph_reg_model = nsl.keras.GraphRegularization(base_model,
                                                graph_reg_config)
graph_reg_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
#graph_reg_model.fit(train_dataset, epochs=200, verbose=1)

In [29]:
graph_reg_model.fit(myTrain.batch(128), epochs=200, verbose=1, validation_data=myTest.batch(128),
          callbacks=[TensorBoard(log_dir='/tmp/regularization')])

Epoch 1/200


2023-03-22 11:16:54.701992: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2023-03-22 11:16:54.702035: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2023-03-22 11:16:54.702087: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/GraphRegularization/graph_loss/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/GraphRegularization/graph_loss/Reshape:0", shape=(None, 7), dtype=float32), dense_shape=Tensor("gradient_tape/GraphRegularization/graph_loss/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


      3/Unknown - 2s 61ms/step - loss: 2.0315 - accuracy: 0.1493 - scaled_graph_loss: 0.0073

2023-03-22 11:16:56.263658: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2023-03-22 11:16:56.263692: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2023-03-22 11:16:56.329523: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2023-03-22 11:16:56.332359: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2023-03-22 11:16:56.336794: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /tmp/regularization/train/plugins/profile/2023_03_22_11_16_56
2023-03-22 11:16:56.340140: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to /tmp/regularization/train/plugins/profile/2023_03_22_11_16_56/5bd49021e10b.trace.json.gz
2023-03-22 11:16:56.342619: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /tmp/regularization/train/plugins/profile/2023_0

5/5 [==============================] - 2s 178ms/step - loss: 2.0350 - accuracy: 0.1445 - scaled_graph_loss: 0.0071 - val_loss: 1.9214 - val_accuracy: 0.2493
Epoch 2/200
5/5 [==============================] - 0s 80ms/step - loss: 2.0045 - accuracy: 0.1628 - scaled_graph_loss: 0.0055 - val_loss: 1.9155 - val_accuracy: 0.2756
Epoch 3/200
5/5 [==============================] - 0s 82ms/step - loss: 1.9872 - accuracy: 0.1954 - scaled_graph_loss: 0.0046 - val_loss: 1.9098 - val_accuracy: 0.2909
Epoch 4/200
5/5 [==============================] - 0s 93ms/step - loss: 1.9716 - accuracy: 0.1785 - scaled_graph_loss: 0.0039 - val_loss: 1.9043 - val_accuracy: 0.3047
Epoch 5/200
5/5 [==============================] - 0s 85ms/step - loss: 1.9698 - accuracy: 0.1719 - scaled_graph_loss: 0.0039 - val_loss: 1.9001 - val_accuracy: 0.3052
Epoch 6/200
5/5 [==============================] - 0s 85ms/step - loss: 1.9309 - accuracy: 0.1966 - scaled_graph_loss: 0.0035 - val_loss: 1.8965 - val_accuracy: 0.3029
Epo